## Video Multi-Object Tracking Inference using Online Endpoints

This sample shows how deploy `video-multi-object-tracking` type models to an online endpoint for inference.

### Task
`video-multi-object-tracking` task monitors multiple objects as they move. The goal is to identify and locate objects of interest in each frame and then associate them across frames to keep track of their movements over time. The output will be assigned boxes with their top-left and bottom-right coordinates along with instance id, box label and confidence score to video frames.
 
### Model
Models that can perform the `video-multi-object-tracking` task are tagged with `video-multi-object-tracking`. We will use the `bytetrack_yolox_x_crowdhuman_mot17-private-half` model in this notebook. If you opened this notebook from a specific model card, remember to replace the specific model name.

### Inference data
We will use an [online video link](https://github.com/open-mmlab/mmtracking/blob/master/demo/demo.mp4)  for multi-object tracking.


### Outline
1. Setup pre-requisites
2. Pick a model to deploy
3. Prepare data for inference
4. Deploy the model to an online endpoint for real time inference
5. Test the endpoint
6. Visualize output
7. Clean up resources - delete the online endpoint

### 1. Setup pre-requisites
* Install dependencies
* Connect to AzureML Workspace. Learn more at [set up SDK authentication](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication?tabs=sdk). Replace  `<WORKSPACE_NAME>`, `<RESOURCE_GROUP>` and `<SUBSCRIPTION_ID>` below.
* Connect to `azureml` system registry

In [ ]:
! pip install azure-ai-ml>=1.23.1
! pip install azure-identity==1.13.0

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import (
    DefaultAzureCredential,
    InteractiveBrowserCredential,
)
import time

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()

try:
    workspace_ml_client = MLClient.from_config(credential)
    subscription_id = workspace_ml_client.subscription_id
    resource_group = workspace_ml_client.resource_group_name
    workspace_name = workspace_ml_client.workspace_name
except Exception as ex:
    print(ex)
    # Enter details of your AML workspace
    subscription_id = "<SUBSCRIPTION_ID>"
    resource_group = "<RESOURCE_GROUP>"
    workspace_name = "<AML_WORKSPACE_NAME>"
workspace_ml_client = MLClient(
    credential, subscription_id, resource_group, workspace_name
)

# The models, fine tuning pipelines and environments are available in the AzureML system registry, "azureml"
registry_ml_client = MLClient(
    credential,
    subscription_id,
    resource_group,
    registry_name="azureml",
)
# Generating a unique timestamp that can be used for names and versions that need to be unique
timestamp = str(int(time.time()))

### 2. Pick a model to deploy

Browse models in the Model Catalog in the AzureML Studio, filtering by the `video-multi-object-tracking` task. In this example, we use the `bytetrack_yolox_x_crowdhuman_mot17-private-half` model. If you have opened this notebook for a different model, replace the model name accordingly. This is a pre-trained model and may not give correct prediction for your dataset. We strongly recommend to finetune this model on a down-stream task to be able to use it for predictions and inference. Please refer to the [video multi-object tracking finetuning notebook](../../finetune/video-multi-object-tracking/mmtracking-video-multi-object-tracking.ipynb).

In [ ]:
model_name = "bytetrack_yolox_x_crowdhuman_mot17-private-half"
foundation_model = registry_ml_client.models.get(name=model_name, label="latest")
print(
    f"Using model name: {foundation_model.name}, version: {foundation_model.version}, id: {foundation_model.id} for inferencing"
)

### 3. Deploy the model to an online endpoint for real time inference
Online endpoints give a durable REST API that can be used to integrate with applications that need to use the model.

In [ ]:
import time
from azure.ai.ml.entities import ManagedOnlineEndpoint, ManagedOnlineDeployment

# Endpoint names need to be unique in a region, hence using timestamp to create unique endpoint name
timestamp = int(time.time())
online_endpoint_name = "video-mot-" + str(timestamp)
# Create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Online endpoint for "
    + foundation_model.name
    + ", for video-multi-object-tracking task",
    auth_mode="key",
)
workspace_ml_client.begin_create_or_update(endpoint).wait()

In [ ]:
from azure.ai.ml.entities import OnlineRequestSettings, ProbeSettings

deployment_name = "video-mot-mlflow-deploy"

print(foundation_model.id)
print(online_endpoint_name)
print(deployment_name)

# Create a deployment
demo_deployment = ManagedOnlineDeployment(
    name=deployment_name,
    endpoint_name=online_endpoint_name,
    model=foundation_model.id,
    instance_type="Standard_NC6s_v3",  # Use GPU instance type only for MOT
    instance_count=1,
    request_settings=OnlineRequestSettings(
        max_concurrent_requests_per_instance=1,
        request_timeout_ms=90000,
        max_queue_wait_ms=500,
    ),
    liveness_probe=ProbeSettings(
        failure_threshold=49,
        success_threshold=1,
        timeout=299,
        period=180,
        initial_delay=180,
    ),
    readiness_probe=ProbeSettings(
        failure_threshold=10,
        success_threshold=1,
        timeout=10,
        period=10,
        initial_delay=10,
    ),
)
workspace_ml_client.online_deployments.begin_create_or_update(demo_deployment).wait()
endpoint.traffic = {deployment_name: 100}
workspace_ml_client.begin_create_or_update(endpoint).result()

### 4. Test the endpoint

We will fetch some sample data from the test dataset and submit to online endpoint for inference.

In [ ]:
demo_deployment = workspace_ml_client.online_deployments.get(
    name=deployment_name,
    endpoint_name=online_endpoint_name,
)

# Get the details for online endpoint
endpoint = workspace_ml_client.online_endpoints.get(name=online_endpoint_name)

# Existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)
print(demo_deployment)


We will use an [online video link](https://github.com/open-mmlab/mmtracking/blob/master/demo/demo.mp4)  for multi-object tracking.

In [ ]:
# Create request json
import json

sample_video_link = "https://github.com/open-mmlab/mmtracking/raw/master/demo/demo.mp4"

request_json = {"input_data": {"columns": ["video"], "data": [sample_video_link]}}
request_file_name = "sample_request_data.json"

with open(request_file_name, "w") as request_file:
    json.dump(request_json, request_file)

In [ ]:
# Score the sample_score.json file using the online endpoint with the azureml endpoint invoke method
response = workspace_ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name=demo_deployment.name,
    request_file=request_file_name,
)
print(f"raw response: {response}\n")

### 5. Visualize the Results

In [ ]:
!pip install opencv-python-headless
!pip install mmcv-full==1.7.1

In [ ]:
import cv2
import mmcv
import json

img_frames = mmcv.VideoReader(sample_video_link)
predictions = json.loads(response)
assert len(img_frames) == len(predictions)

In [ ]:
def draw_bbox_on_image(img, track_bbox):
    x0, y0, x1, y1 = (
        track_bbox["topX"],
        track_bbox["topY"],
        track_bbox["bottomX"],
        track_bbox["bottomY"],
    )
    x0, y0, x1, y1 = int(x0), int(y0), int(x1), int(y1)
    instance_id = track_bbox["instance_id"]
    text = f"ID: {instance_id}"
    cv2.putText(img, text, (x0, y0), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1)
    cv2.rectangle(img, (x0, y0), (x1, y1), color=(0, 0, 0), thickness=2)


visualized_results = []
for img, prediction in zip(img_frames, predictions):
    track_bboxes = prediction["track_bboxes"]
    for track_bbox in track_bboxes:
        draw_bbox_on_image(img, track_bbox["box"])
    visualized_results.append(img)

In [ ]:
from PIL import Image
from IPython.display import display, clear_output
from time import sleep

fps = 10  # frames per second, for most videos fps=30, pls change it according to your video
for img_array in visualized_results:
    display(Image.fromarray(img_array[:, :, ::-1]))
    sleep(1.0 / fps)
    clear_output(wait=True)

### 6. Clean up resources - delete the online endpoint
Don't forget to delete the online endpoint, else you will leave the billing meter running for the compute used by the endpoint.

In [ ]:
workspace_ml_client.online_endpoints.begin_delete(name=online_endpoint_name).wait()